In [299]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from scipy import stats
from pprint import pprint
import sys
from colorama import Style, Fore
import warnings
import time 
import datetime as dt 



warnings.filterwarnings(action = 'ignore')

rc = {
    "axes.facecolor": "#F6F6F6",
    "figure.facecolor": "#F6F6F6",
    "axes.edgecolor": "#000000",
    "grid.color": "#EBEBE7",
    "font.family": "serif",
    "axes.labelcolor": "#000000",
    "xtick.color": "#000000",
    "ytick.color": "#000000",
    "grid.alpha": 0.4
}

sns.set(rc=rc)


red = Style.BRIGHT + Fore.RED
blu = Style.BRIGHT + Fore.BLUE
mgt = Style.BRIGHT + Fore.MAGENTA
gld = Style.BRIGHT + Fore.YELLOW
res = Style.RESET_ALL

In [237]:
data = pd.read_csv("C:/Users/whileduck/Downloads/2023빅콘테스트_정형데이터분석분야_어드밴스드리그_데이터셋/2023빅콘테스트_어드밴스드리그_예술의전당.csv")

In [259]:
class DataExploratioin:
    '''
    데이터 탐색 시 사용 가능한 Class 

    기존 존재하는 프레임워크들을 이용하여 자주 이용하는 프레임워크들을 활용하여 나만의 분석 툴을 만들려고 함 

    데이터 요약, 결측값 처리 등의 내용이 담겨있는 class 
    '''

    def __init__(self, data):
        self.data = data

    def summarize(self):
        '''
        데이터를 초창기에 요약해주는 method
        '''

        cols = self.data.columns

        size = round(sys.getsizeof(self.data) / 1024 ** 2, 2)

        print(f'data size : {size}MB')

        self.result = pd.DataFrame()

        self.result['Dtype'] = self.data.dtypes.values
        self.result['Count'] = self.data.count().values
        self.result['Nunique'] = self.data.nunique().values
        self.result['Missing value'] = self.data.isna().sum().values
        self.result['Missing %'] = [str(round(
            missing / len(self.data), 2) * 100) + '%' for missing in self.result['Missing value']]
        self.result['Most Freq Value'] = self.data.mode().iloc[0].values

        freq_prop = []

        for i, col in enumerate(cols):

            raw_data = self.data.loc[~self.data[col].isna(), col]
            freq_value = self.result['Most Freq Value'].iloc[i]

            prop = np.mean(
                np.array(raw_data == freq_value)
            )

            prop_str = str(round(np.mean(prop) * 100, 1)) + '%'

            if prop_str == 'nan%':
                freq_prop.append(self.result['Missing %'].iloc[i])
            else:
                freq_prop.append(prop_str)

        self.result['Most Freq Value %'] = freq_prop

        self.result['Min'] = self.data.describe(include='all').T['min'].values
        self.result['Max'] = self.data.describe(include='all').T['max'].values
        self.result['Mean'] = self.data.describe(
            include='all').T['mean'].values
        self.result['Median'] = self.data.describe(
            include='all').T['50%'].values
        
        memory = (self.data.memory_usage(deep = True) // 1024 **2).values[1:] # index 의 usage 는 제외하고 보자 

        
        self.result['MB'] = [str(m) + ' mb' for m in memory]
        self.result = self.result.set_index(cols)

        self.result = self.result.fillna('-')

        display(self.result)
    
    
    def progress_bar(self,iterable, total_blocks = 10):
        
        total_items = len(iterable)
        block_size = total_items // total_blocks
        
        for i, item in enumerate(iterable, start=1):
            if i % block_size == 0 or i == total_items:
                progress = (i / total_items) * 100
                blocks = int(progress / (100 / total_blocks))
                empty_blocks = total_blocks - blocks
                progress_bar = '■' * blocks + '▢' * empty_blocks
                print(f"\rProgress: [{progress_bar}] {progress:.2f}%", end='', flush=True)
            yield item
            time.sleep(0.0000001)
    
    def reduce_size(self):
                
        original_size = round(sys.getsizeof(self.data) / 1024 ** 2,2)
        
        df = self.data.copy()
        
        for col in self.progress_bar(df.columns):
            
            dtp = df[col].dtype
            
            if dtp == 'object':
                df[col] = df[col].astype('category')
            else: # numeric type이면 
                
                if min(df[col]) >= 0 : # 부호가 없다면 unit 으로 변경해줘도 된다.
                    max_value = max(df[col])
                    
                    bits = [8,16,32,64]
                    
                    for bit in bits: # 최소한의 비트로 표현 될 수 있게 dtype 변경 
                        if max_value < 2 ** bit:
                            # 결측치가 있는 경우 astype 으로 변경하지 못하니 결측치를 채워준 후 변경하고 다시 결측치를 채우자 
                            df[col] = df[col].fillna(2 ** bit - 1)
                            df[col] = df[col].astype(f'uint{bit}')
                            df[col] = df[col].replace(2 ** bit - 1, np.NaN)
                            break
                        
                else: # 부호가 있다면 int type 으로 바꿔주자 
                    
                    max_value = max(abs(min(df[col])), max(df[col]))
                    
                    bits = [8,16,32,64]
                    
                    for bit in bits:
                        if max_value < 2 ** bit:
                            df[col] = df[col].fillna(2 ** bit - 1)
                            df[col] = df[col].astype(f'int{bit}')
                            df[col] = df[col].replace(2 ** bit - 1, np.NaN)
                            break
                        
        print('\n')
                        
        after_size = round(sys.getsizeof(df) / 1024 ** 2,2)
        
        # 바꾼 후 결과 보여주기 
        after = DataExploratioin(df)
        after.summarize()
        
        print(f'\n {original_size}MB -> {after_size}MB')
            
        return df
                        
            
                

In [260]:
ep = DataExploratioin(data)

In [261]:
ep.summarize()

data size : 1501.26MB


,Dtype,Count,Nunique,Missing value,Missing %,Most Freq Value,Most Freq Value %,Min,Max,Mean,Median,MB
age,float64,620685,8,1300183,68.0%,40.0,23.9%,10.0,80.0,42.39516,40.0,14 mb
gender,object,621154,2,1299714,68.0%,F,65.1%,-,-,-,-,74 mb
membership_type_1,object,621154,3,1299714,68.0%,무료,49.2%,-,-,-,-,90 mb
membership_type_2,object,487653,5,1433215,75.0%,무료,49.4%,-,-,-,-,84 mb
membership_type_3,object,171859,4,1749009,91.0%,무료,43.5%,-,-,-,-,67 mb
membership_type_4,object,30193,3,1890675,98.0%,그린,77.5%,-,-,-,-,60 mb
membership_type_5,object,1282,1,1919586,100.0%,그린,100.0%,-,-,-,-,58 mb
membership_type_6,float64,0,0,1920868,100.0%,-,100.0%,-,-,-,-,14 mb
tran_date,int64,1920868,1689,0,0.0%,20230519,0.4%,20181101.0,20230703.0,20209376.555199,20210919.0,14 mb
tran_time,int64,1920868,1440,0,0.0%,1401,0.5%,0.0,2359.0,1401.515166,1419.0,14 mb



# **💡insight** 

데이터 사이즈 : 1501.26 MB


0. 전체 데이터 데이터 중 빈좌석은 68% 에 육박합니다. 코로나로 인한 대면 거리두기 등에 따라 상이하겠지만 age 와 gender 의 경우가 missing value 인 경우가 빈좌석입니다.

1. 연령대는 bining 된 형태로 존재합니다 10대~80대 까지 있습니다. 현재 테이블에선 볼 수는 없지만 30~50대 연령이 70% 가량 있으며 10대, 20대의 이용은 13% 됩니다. 10대의 이용비율은 0.8% 밖에 안됩니다.

2. 성별은 여성의 비율이 65%, 남성의 비율이 35%입니다

3. 각 멤버십 타입 별 멤버십 가지수의 최대치는 최대 5개인 것으로 보입니다. 멤버십 타입 5, 6 까지 가지고 결제한 고객은 많이 있지 않습니다.

4. 유니크한 좌석의 개수는 3525개입니다.

5. 가격들의 유니크한 개수는 208 개입니다.

6. 예매 취소 비율은 26.6% 입니다. (2 는 예매 취소하지 않음, 0은 예매 취소하지 않음)

7. 할인을 받은 경우 초대권이 46.8%로 가장 많았습니다. 할인을 받지 않은 경우는 ‘일반’으로 맵핑 되는데 할인 받지 않은 경우는 12%입니다. (오차 존재함)

8. 공연 장르의 유니크한 개수는 15개입니다. 그 중 교향곡이 28.9% 로 가장 빈도가 높았습니다. 

9. 공연 장소는 유니크한 값이 3개이며, 그 중 57% 는 콘서트 홀에서 공연하였습니다.


### **데이터 용량 줄이기**

In [262]:
df = ep.reduce_size()

Progress: [■■■■■■■■■■] 100.00%

data size : 123.44MB


,Dtype,Count,Nunique,Missing value,Missing %,Most Freq Value,Most Freq Value %,Min,Max,Mean,Median,MB
age,float64,620685,8,1300183,68.0%,40.0,23.9%,10.0,80.0,42.39516,40.0,14 mb
gender,category,621154,2,1299714,68.0%,F,65.1%,-,-,-,-,1 mb
membership_type_1,category,621154,3,1299714,68.0%,무료,49.2%,-,-,-,-,1 mb
membership_type_2,category,487653,5,1433215,75.0%,무료,49.4%,-,-,-,-,1 mb
membership_type_3,category,171859,4,1749009,91.0%,무료,43.5%,-,-,-,-,1 mb
membership_type_4,category,30193,3,1890675,98.0%,그린,77.5%,-,-,-,-,1 mb
membership_type_5,category,1282,1,1919586,100.0%,그린,100.0%,-,-,-,-,1 mb
membership_type_6,float64,0,0,1920868,100.0%,-,100.0%,-,-,-,-,14 mb
tran_date,uint32,1920868,1689,0,0.0%,20230519,0.4%,20181101.0,20230703.0,20209376.555199,20210919.0,7 mb
tran_time,uint16,1920868,1440,0,0.0%,1401,0.5%,0.0,2359.0,1401.515166,1419.0,3 mb



 1501.26MB -> 123.44MB


# **💡insight** 

데이터 타입을 변경하는 것만으로도 용량을 90% 가량 줄일 수 있었습니다.

데이터 타입을 변경하여도 데이터의 통계량에는 변화가 없는 모습을 볼 수 있습니다.

데이터의 메모리 사용량을 감소시켜 이후 과정에서 연산속도의 향상을 기대 할 수 있을 것 같습니다.

### **시계열 데이터 처리**

| 컬럼ID | 컬럼명 | 비고 |
|-------|-------|-------|
| tran_date   |예매 거래 일자    | YYYYMMDD   |
| tran_time   |예매 거래 시간  |HHMM   |
| play_date   |공연 시작 날짜  |YYYYMMDD   |
| play_st_time   |공연 시작 시간  |HHMM   |
| pre_open_date    |선예매시작일  |YYYYMMDD   |
| open_date    |예매시작일  |YYYYMMDD   |


시계열 데이터를 데이터 타임 형태로 변경해주겠습니다.

pandas 의 pd.to_datetime 을 이용하면 쉽게 변경 할 수 있습니다.

In [306]:
def change_datetiime(df, cols):
    
    format_type = {
        'tran_date' : 'YYYYMMDD',
        'tran_time' : 'HHMM',
        'play_date' : 'YYYYMMDD',
        'play_st_time' : 'HHMM',
        'pre_open_date' : 'YYYYMMDD',
        'open_date' : 'YYYYMMDD'
    }
    
    data = df.copy()
    
    for col in cols:
        
        if format_type[col] == 'YYYYMMDD':
            format = '%Y%m%d'
            
            data[col] = pd.to_datetime(data[col], format = format)
            
        if format_type[col] == 'HHMM':
            format = '%H%M'

            fill_col = data[col].apply(lambda x: str(x).zfill(4)) # 문자열 형태로 변경 후 4글자인 HHMM 형태로 맞춰줍니다.
                                                            # 현재 데이터에서는 오전 9시 30분인 경우 930 으로 써있습니다.
            data[col] = pd.to_datetime(fill_col, format = format).dt.time
    return data

In [308]:
time_cols = ['tran_date' , 'tran_time','play_date', 'play_st_time','pre_open_date','open_date']

df = change_datetiime(df,  cols = time_cols )

display(df[time_cols])

,tran_date,tran_time,play_date,play_st_time,pre_open_date,open_date
0,2022-01-14,15:12:00,2022-02-04,20:00:00,2022-01-14,2022-01-15
1,2021-10-03,15:50:00,2021-10-09,20:00:00,NaT,2021-08-23
2,2022-09-17,15:20:00,2022-11-05,20:00:00,2022-09-11,2022-09-11
3,2019-06-30,11:32:00,2019-07-31,20:00:00,NaT,2019-07-01
4,2022-02-06,16:15:00,2022-03-02,19:30:00,2022-01-09,2022-01-10
...,...,...,...,...,...,...
1920863,2019-06-14,09:20:00,2019-07-06,14:00:00,2019-05-28,2019-05-31
1920864,2022-12-24,12:23:00,2023-04-08,17:00:00,NaT,2022-12-23
1920865,2021-06-14,01:29:00,2021-06-19,20:00:00,NaT,2021-05-03
1920866,2021-11-23,16:17:00,2021-11-27,20:00:00,2021-10-29,2021-10-29


In [310]:
df.to_csv('2023-08-15 가공 데이터.csv', index = False)